In [28]:
import scrapy
from bs4 import BeautifulSoup

In [22]:
# 'img_src': response.css('div.td-post-content img::attr(src)').getall(),
import pymongo

client = pymongo.MongoClient()
db = client["news"]
coll = db["sebenarnya_test3"]



In [ ]:
coll.find_one()

In [41]:
content = coll.find_one()['content_html']
soup = BeautifulSoup(content[0], 'html.parser')
[line.strip() for line in soup.text.split('\n') if line.strip() != '']

['SEBENARNYA:',
 'Pihak Hospital Segamat ingin merujuk kepada mesej berangkai yang tular di aplikasi WhatsApp berkaitan hebahan maklumat Kes Positif COVID-19 Daerah Segamat yang dikeluarkan oleh Hospital Segamat.',
 'Untuk makluman, Hospital Segamat tidak membuat sebarang hebahan berkaitan Kes Positif COVID-19 di setiap kawasan di Daerah Segamat. Pihak Hospital Segamat juga ingin menafikan bahawa sebarang hebahan maklumat berkaitan kes COVID-19 di daerah Segamat yang dikatakan sumbernya diperolehi daripada Hospital Segamat adalah tidak benar.',
 'SUMBER:',
 'Facebook Rasmi Jabatan Kesihatan Negeri Johor',
 'MAINKAN PERANAN ANDA!',
 'Sekiranya anda mempunyai maklumat mengenai berita tidak ditentusah yang melibatkan kepentingan awam mahupun negara,',
 'Salurkan Kepada Kami',
 'TIDAK PASTI JANGAN KONGSI']

In [25]:
len(list(coll.find()))

30

In [17]:
# coll.drop()

{'_id': ObjectId('5e86081fde9d87570b10b145')}